In [1]:
# file dependencies
from tacc_stats.pickler.job_stats import Job
import cPickle as pickle
import argparse
from os import listdir

In [2]:
# data manipulation dependencies
import pandas as pd
import numpy as np
import datetime as dt

In [3]:
# pickle file with job data
pickle_file = open('/oasis/projects/nsf/sys200/tcooper/xsede_stats/comet_pickles/2016-08-21/3864417', 'rb')

In [4]:
# open pickle file
# extract job
jobid = pickle.load(pickle_file)
pickle_file.close()

In [5]:
# general stats
hosts = jobid.hosts
start = pd.to_datetime(round(jobid.start_time), unit='s').time()
end = pd.to_datetime(round(jobid.end_time), unit='s').time()
numCycles = len(jobid.times) + 2

In [25]:
type_avgs = {}
times = [start]

for host_name, host in jobid.hosts.iteritems():
    typ='all'
    dev='all'
    
    if 'all' in typ: typ_keys = host.stats.keys()
    else: typ_keys = typ
        
    for time in host.times:
        times.append( pd.to_datetime(round(time), unit='s').time() )
    
    for type_name in typ_keys:
        type_data = host.stats[type_name]
        type_avgs[type_name] = []
        
        if 'all' in dev: dev_keys = type_data.keys()
        else: dev_keys = dev
            
        for dev_name in dev_keys:                    
            type_avgs[type_name].append( [np.mean(row).round(decimals=3)
                                   for row in type_data[dev_name]] )
times.append(end)

In [75]:
all_stats = { time:{ stat:[] for stat in type_avgs } for time in times }

In [78]:
for type_avg,data in type_avgs.iteritems():
    for dev in range( len(data) ):
        all_stats[dt.time(21, 59, 44)][type_avg] .append(  data[dev][0])
        all_stats[dt.time(22, 9, 45)] [type_avg] .append(   data[dev][1])   
        all_stats[dt.time(22, 19, 45)][type_avg] .append(  data[dev][2]) 
        all_stats[dt.time(22, 29, 45)][type_avg] .append(  data[dev][3])
        all_stats[dt.time(22, 39, 45)][type_avg] .append(  data[dev][4])
        all_stats[dt.time(22, 49, 45)][type_avg] .append(  data[dev][5])   
        all_stats[dt.time(0, 38, 3)]  [type_avg] .append(    data[dev][6])

In [93]:
for time,stat in all_stats.iteritems():
    for stat,avgList in stat.iteritems():
        print avgList
        print np.mean(avgList)
        print np.std(avgList)
        print np.max(avgList)
        print np.min(avgList)

[14530.5, 6203.0]
10366.75
4163.75
14530.5
6203.0
[7273413.2379999999]
7273413.238
0.0
7273413.238
7273413.238
[84921896477.0, 41223124862.0, 84938078158.25, 41240023276.75]
63080780693.5
21849208190.0
84938078158.2
41223124862.0
[146468728162.25, 143858979108.75, 122196497937.75, 121257826858.0, 168393234892.5, 137471650355.5, 167446354185.0, 138835602647.75, 142857418276.75, 142575786114.25, 27986235072.75, 30081253501.5, 26389588751.5, 26807688613.5, 26233693128.25, 26654691529.0, 26039719997.5, 25713837148.75, 31976417778.5, 25582746512.5, 27771855370.5, 28525175318.0, 125034313680.0, 114731447725.25]
83370447611.1
57072014806.6
168393234892.0
25582746512.5
[139039095791.83301, 200365150571.0]
169702123181.0
30663027389.6
200365150571.0
139039095792.0
[528753632.63599998, 511655970.0, 233917232.90900001, 176392302.90900001, 112190434655.909, 110417269096.0, 110171723744.72701, 111109057704.091, 544848111157.18201, 111112315811.0, 545959472930.27301, 111147786331.364, 537924469966.2

In [19]:
# merge all data
df = pd.DataFrame( all_stats, index=times )

In [20]:
df

,Host,block,cpu,ib,ib_ext,ib_sw,intel_hsw,intel_hsw_cbo,intel_hsw_hau,intel_hsw_imc,...,net,nfs,numa,osc,proc,ps,sysv_shm,tmpfs,vfs,vm
22:09:45,comet-03-09,8387872.766234,24613.755102,NaN,3.083450e+09,7.705510e+09,7.272910e+11,2.459960e+11,1.375834e+12,4.022456e+11,...,2.642803e+08,50080.225902,6712514.785714,33875.292857,6.081344e+17,3790411.44898,NaN,2,411487.190476,20038704.285714
22:19:45,comet-03-09,8387872.766234,24613.755102,NaN,3.083450e+09,7.705510e+09,7.272910e+11,2.459960e+11,1.375834e+12,4.022456e+11,...,2.642803e+08,50080.225902,6712514.785714,33875.292857,6.081344e+17,3790411.44898,NaN,2,411487.190476,20038704.285714
22:29:45,comet-03-09,8387872.766234,24613.755102,NaN,3.083450e+09,7.705510e+09,7.272910e+11,2.459960e+11,1.375834e+12,4.022456e+11,...,2.642803e+08,50080.225902,6712514.785714,33875.292857,6.081344e+17,3790411.44898,NaN,2,411487.190476,20038704.285714
22:39:45,comet-03-09,8387872.766234,24613.755102,NaN,3.083450e+09,7.705510e+09,7.272910e+11,2.459960e+11,1.375834e+12,4.022456e+11,...,2.642803e+08,50080.225902,6712514.785714,33875.292857,6.081344e+17,3790411.44898,NaN,2,411487.190476,20038704.285714
22:49:45,comet-03-09,8387872.766234,24613.755102,NaN,3.083450e+09,7.705510e+09,7.272910e+11,2.459960e+11,1.375834e+12,4.022456e+11,...,2.642803e+08,50080.225902,6712514.785714,33875.292857,6.081344e+17,3790411.44898,NaN,2,411487.190476,20038704.285714


In [23]:
df = df[["intel_hsw","ib","llite",
         # "ib_sw","ib_ext", "mdc","osc",
         "lnet", "block","cpu","mem", "net",
         "nfs","numa","proc","sysv_shm","tmpfs"]]

In [24]:
# select and rename stats
df = df.rename( columns={"index": "Job Name",
                         "intel_hsw": "Intel Haswell Processor (per core)",
                         "ib": "Infiniband Usage",
                         #"ib_sw": "Infiniband Usage",
                         #"ib_ext": "Infiniband Usage",
                         "llite": "Lustre Filesystem Usage (per mount)",
                         "lnet": "Lustre Network Usage",
                         #"mdc": "Lustre Network Usage",
                         #"osc": "Lustre Filesystem Usage",
                         "block": "Block Device Statistics (per device)",
                         "cpu": "Scheduler Accounting (per CPU)",
                         "mem": "Memory Usage",
                         "net": "Network Device Usage (per device)",
                         "nfs": "NFS System Usage",
                         "numa": "NUMA Statistics (per socket)",
                         "proc": "Process Specific Data",
                         "sysv_shm": "System Shared Memory Segment Usage",
                         "tmpfs": "RAM-Backed Filesystem Usage (per mount)"
                        })

In [25]:
# reorder columns
df = df.reindex(columns=[
   "Block Device Statistics (per device)",
   "Intel Haswell Processor (per core)",
   "Infiniband Usage",
   #"Infiniband Usage",
   #"Infiniband Usage",
   "Lustre Filesystem Usage (per mount)",
   "Lustre Network Usage",
   #"Lustre Network Usage",
   #"Lustre Filesystem Usage",
   "Memory Usage",
   "Network Device Usage (per device)",
   "NFS System Usage",
   "NUMA Statistics (per socket)",
   "Process Specific Data",
   "RAM-Backed Filesystem Usage (per mount)",
   "Scheduler Accounting (per CPU)",
   "System Shared Memory Segment Usage"])
df = df.dropna(axis=1)

In [26]:
df.head()

,Block Device Statistics (per device),Intel Haswell Processor (per core),Lustre Filesystem Usage (per mount),Lustre Network Usage,Memory Usage,Network Device Usage (per device),NFS System Usage,NUMA Statistics (per socket),Process Specific Data,RAM-Backed Filesystem Usage (per mount),Scheduler Accounting (per CPU)
22:09:45,8387872.766234,7.272910e+11,5.296619e+08,3.367214e+09,1.514963e+10,2.642803e+08,50080.225902,6712514.785714,6.081344e+17,2,24613.755102
22:19:45,8387872.766234,7.272910e+11,5.296619e+08,3.367214e+09,1.514963e+10,2.642803e+08,50080.225902,6712514.785714,6.081344e+17,2,24613.755102
22:29:45,8387872.766234,7.272910e+11,5.296619e+08,3.367214e+09,1.514963e+10,2.642803e+08,50080.225902,6712514.785714,6.081344e+17,2,24613.755102
22:39:45,8387872.766234,7.272910e+11,5.296619e+08,3.367214e+09,1.514963e+10,2.642803e+08,50080.225902,6712514.785714,6.081344e+17,2,24613.755102
22:49:45,8387872.766234,7.272910e+11,5.296619e+08,3.367214e+09,1.514963e+10,2.642803e+08,50080.225902,6712514.785714,6.081344e+17,2,24613.755102


In [ ]:
df.count()

### Computing Averages

In [ ]:
# copy df to alter values
means_df = df

# compute avg of non-nan values
for row in range(0, len(df.index)):
    for col in range(0, len(df.columns)):
        cell = df.iat[row,col]
        if ( type(cell) == np.ndarray ):
            means_df.iat[row,col] = np.mean(cell)

means_df.head()

In [ ]:
# focus on columns with least non-nan values
actual_df = pd.DataFrame(means_df[['Job Name', 'Intel Haswell Processor (per core)', 
                                   'Scheduler Accounting (per CPU)']].dropna())
actual_df = actual_df.reset_index()[['Job Name', 'Intel Haswell Processor (per core)',
                                     'Scheduler Accounting (per CPU)']]

In [ ]:
actual_df.head()

In [ ]:
# Add column for number of times stats collected
# And column for 